In [ ]:
!pip install langchain openai wikipedia tiktoken neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 kB 17.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=e5d21b29a0c385fd1245fcc98afda8277528002c2159f7dadf04b420612d62c1
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
  Created wheel for neo4j: filename=n

In [ ]:
import os

from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

os.environ["OPENAI_API_KEY"] = "***********************"

In [ ]:
raw_documents = WikipediaLoader(query="Hardeep Singh Nijjar").load()
print(raw_documents)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=20
)

documents = text_splitter.split_documents(raw_documents)

[Document(page_content='Hardeep Singh Nijjar (11 October 1977 – 18 June 2023) was a Canadian  Sikh separatist leader involved with the Khalistan movement, which calls for an independent Sikh state.Born in India, Nijjar migrated to Canada in the mid-1990s. Sikh organizations viewed Nijjar as a human rights activist, while the Indian government accused him of being a criminal and terrorist affiliated with the militant Khalistan Tiger Force, and sought his arrest. Nijjar and his supporters rejected these allegations, saying he advocated peaceful means for creation of Khalistan. Nijjar gained prominence in 2019, when he became the leader of Guru Nanak Sikh Gurudwara (temple) in Surrey, British Columbia, and became an advocate of Sikh separatism. Nijjar was also associated with Sikhs for Justice (SFJ), and spearheaded the group\'s Khalistan Referendum 2020 campaign.On 18 June 2023, Nijjar was shot and killed in the parking lot of a Sikh temple in British Columbia. On 18 September 2023, Cana

In [ ]:
for d in documents:
    del d.metadata["summary"]

In [ ]:
url = "neo4j+s://0ceceb70.databases.neo4j.io"
username = "neo4j"
password = "***********************"

neo4j_vector = Neo4jVector.from_documents(
    documents, OpenAIEmbeddings(), url=url, username=username, password=password
)

In [ ]:
query = "Where did Nijjar grow up?"

results = neo4j_vector.similarity_search(query, k=1)
print(results[0].page_content)

Nijjar is a surname found in India. Notable people with the surname include:

Aron Nijjar (born 1994), English cricketer
Rob Nijjar (born 1967), Canadian politician
Sunny Singh Nijjar, Indian actor
Surinder Singh Nijjar (1949–2021), Indian judge
Hardeep Singh Nijjar (1977–2023), Canadian Sikh separatist leader


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), chain_type="stuff", retriever=neo4j_vector.as_retriever()
)

In [ ]:
query = "Who is nijjar?"

chain(
    {"question": query},
    return_only_outputs=True,
)

{'answer': "Nijjar is a surname found in India. Notable people with the surname include Aron Nijjar (English cricketer), Rob Nijjar (Canadian politician), Sunny Singh Nijjar (Indian actor), Surinder Singh Nijjar (Indian judge), and Hardeep Singh Nijjar (Canadian Sikh separatist leader). Hardeep Singh Nijjar was involved with the Khalistan movement and was shot and killed in 2023. He was accused by the Indian government of being a criminal and terrorist affiliated with the Khalistan Tiger Force. Canada expelled an Indian diplomat in connection with his assassination. India denied involvement and expelled a top Canadian diplomat in retaliation. Nijjar's involvement with the Khalistan Tiger Force included training and financing militants for the organization. He was also associated with Sikhs for Justice and spearheaded the group's Khalistan Referendum 2020 campaign. The Khalistan Tiger Force is a militant outfit of the Khalistan movement and was banned by the Indian government in 2023. I

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(temperature=0), neo4j_vector.as_retriever(), memory=memory
)

Let's now test it out.

In [ ]:
print(qa({"question": "Where was nijjar born"})["answer"])

Nijjar was born in a village in Jalandhar, Punjab, India.


And now a follow-up question.

In [ ]:
print(qa({"question": "Write a 100 word story about nijjar"})["answer"])

Nijjar, a Sikh separatist leader, fought tirelessly for the creation of Khalistan, an independent Sikh state. He migrated to Canada in the mid-1990s, where he became a prominent figure in the local Sikh community. Nijjar led peaceful protests, advocated for human rights, and organized a referendum in support of Khalistan. However, his activism made him a target. Tragically, on a fateful day in June 2023, Nijjar was assassinated outside a Sikh temple in British Columbia. The investigation into his murder revealed potential links to Indian government agents, sparking tensions between Canada and India. Nijjar's legacy as a passionate advocate for Sikh rights lives on.
